In [40]:
from collections import defaultdict
class duet():
    def __init__(self, debug=False):
        self.register = defaultdict(int)
        self.mem = []
        self.loc = 0
        self.sound = 0
        self.received = 0
        self.wait = False
        self.debug = debug
        
    def getVal(self, val):
        if isinstance(val, int):
            return val
        return self.register[val]
    
    @staticmethod
    def parse(string):
        args = string.split()
        ret = [args[0]]
        for arg in args[1:]:
            try:
                ret.append(int(arg))
            except ValueError:
                ret.append(arg)
        return ret
    
    def load(self, string):
        self.mem = list(map(self.parse, string.split('\n')))
    
    def exe(self, *args):
        getattr(self, args[0])(*args[1:])
        if self.debug:
            print(args, ': ', self.register)
        
    def run(self):
        self.wait = False
        while self.loc < len(self.mem) and self.loc >= 0:
            l = self.loc
            self.exe(*self.mem[self.loc])
            if l == self.loc and self.wait == False:
                self.loc += 1
            if self.wait == True:
                return 1
        return 0
    
    def snd(self, x):
        self.sound = self.getVal(x)
        
    def set(self, x, y):
        self.register[x] = self.getVal(y)
    
    def add(self, x, y):
        self.register[x] += self.getVal(y)
        
    def mul(self, x, y):
        self.register[x] *= self.getVal(y)
        
    def mod(self, x, y):
        self.register[x] = self.getVal(x) % self.getVal(y)
        
    def rcv(self, x):
        if self.getVal(x):
            self.received = self.sound
            print(self.received)
            self.loc = -1
    
    def jgz(self, x, y):
        if self.getVal(x) > 0:
            self.loc += self.getVal(y)
       
        

In [41]:
# test
test_inp = """set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2"""

In [42]:
d = duet()
d.load(test_inp)
d.run()
assert(d.received == 4)

4


In [43]:
with open('data/input_18.txt') as fh:
    inp = fh.read().strip()

In [44]:
#part 1
d = duet()
d.load(inp)
d.run()

4601


0

In [49]:
class duet2(duet):
    def __init__(self, pid, name="", **kwargs):
        super().__init__(**kwargs)
        self.register['p'] = pid
        self.name = name
        self.queue = []
        self.send_cnt = 0
        
    def rcv(self, x):
        if len(self.queue) == 0:
            self.wait = True
            return
        self.register[x] = self.queue.pop(0)
        if self.debug:
            print('{} recvs {} | loc {}'.format(self.name, self.register[x], self.loc))
        
    def snd(self, x):
        self.other.queue.append(self.getVal(x))
        self.send_cnt += 1
        if self.debug:
            print('{} sends {} | loc {}'.format(self.name, self.getVal(x), self.loc))


In [50]:
test_inp = """snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d"""

In [52]:
a = duet2(0, debug = True, name='0')
b = duet2(1, debug = False, name='1')
a.other = b
b.other = a
a.load(test_inp)
b.load(test_inp)

while True:
    a_running = a.run()
    b_running = b.run()
    if not len(a.queue) and not len(b.queue):
        break
    if not a_running and not b_running:
        break

print(a.register, a.send_cnt)
print(b.register, b.send_cnt)

0 sends 1 | loc 0
('snd', 1) :  defaultdict(<class 'int'>, {'p': 0})
0 sends 2 | loc 1
('snd', 2) :  defaultdict(<class 'int'>, {'p': 0})
0 sends 0 | loc 2
('snd', 'p') :  defaultdict(<class 'int'>, {'p': 0})
('rcv', 'a') :  defaultdict(<class 'int'>, {'p': 0})
0 recvs 1 | loc 3
('rcv', 'a') :  defaultdict(<class 'int'>, {'p': 0, 'a': 1})
0 recvs 2 | loc 4
('rcv', 'b') :  defaultdict(<class 'int'>, {'p': 0, 'a': 1, 'b': 2})
0 recvs 1 | loc 5
('rcv', 'c') :  defaultdict(<class 'int'>, {'p': 0, 'a': 1, 'b': 2, 'c': 1})
('rcv', 'd') :  defaultdict(<class 'int'>, {'p': 0, 'a': 1, 'b': 2, 'c': 1})
defaultdict(<class 'int'>, {'p': 0, 'a': 1, 'b': 2, 'c': 1}) 3
defaultdict(<class 'int'>, {'p': 1, 'a': 1, 'b': 2, 'c': 0}) 3


In [54]:
#part 2
a = duet2(0, name='0')
b = duet2(1, name='1')
a.other = b
b.other = a
a.load(inp)
b.load(inp)

while True:
    a_running = a.run()
    b_running = b.run()
    if not len(a.queue) and not len(b.queue):
        break
    if not a_running and not b_running:
        break

print(a.name, a.send_cnt, a.register)
print(b.name, b.send_cnt, b.register)

0 6985 defaultdict(<class 'int'>, {'p': -6, 'i': 0, 'a': 114, 'b': 114, 'f': 0})
1 6858 defaultdict(<class 'int'>, {'p': -6, 'i': 0, 'a': 114, 'f': 0, 'b': 114})
